In [5]:
import boto3
import json
from jsonschema import validate
from elasticsearch import Elasticsearch, RequestsHttpConnection
from elasticsearch.helpers import scan
from requests_aws4auth import AWS4Auth
import os

region = 'us-east-1'
service = 'es'
eshost = 'search-world-modelers-dev-gjvcliqvo44h4dgby7tn3psw74.us-east-1.es.amazonaws.com'

session = boto3.Session(region_name=region, profile_name='wmuser')
credentials = session.get_credentials()
credentials = credentials.get_frozen_credentials()
access_key = credentials.access_key
secret_key = credentials.secret_key
token = credentials.token

aws_auth = AWS4Auth(
    access_key,
    secret_key,
    region,
    service,
    session_token=token
)


In [2]:
es = Elasticsearch(
    hosts = [{'host': eshost, 'port': 443}],
    http_auth=aws_auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

In [3]:
index = "wm-dev"

## Document pull for UAZ

In [4]:
query = {
    "aggs" : {
        "categories" : {
            "terms" : { "field" : "category.keyword" } 
        }
    }
}

categories = es.search(index=index, body=query)['aggregations']['categories']['buckets']

In [5]:
categories

[{'key': 'Twelve-Month Eval Docs', 'doc_count': 473},
 {'key': 'November 2019 SSudan Docs', 'doc_count': 405},
 {'key': 'November 2019 Ethiopia Docs', 'doc_count': 255},
 {'key': 'Six-Month Evaluation Documents', 'doc_count': 38},
 {'key': 'Additional Six-Month Eval Docs', 'doc_count': 29},
 {'key': 'Luma-Provided Ethiopia Docs', 'doc_count': 21},
 {'key': 'Migration', 'doc_count': 19}]

In [14]:
results = []

cats = ['Twelve-Month Eval Docs','Six-Month Evaluation Documents','Additional Six-Month Eval Docs'],

for cat in cats:
    query = {
        "query" : {
            "terms" : {
                "category.keyword" : cat
            }
        }
    }

    res = es.search(index=index, body=query, size=1000)['hits']['hits']
    results.extend(res)

In [16]:
len(results) == 473+38+29

True

In [37]:
len(dsmt_docs)

540

In [20]:
dsmt_docs = []
for res in results:
    _id = res['_id']
    source = res['_source']
    source_url = source['source_url']
    stored_url = source['stored_url']
    file_name = source['file_name']
    obj = {"_id": _id, "source_url": source_url, "stored_url": stored_url, "file_name": file_name}
    dsmt_docs.append(obj)

In [36]:
with open('DSMT-E_Document_Summary.jsonl','a') as f:
    for doc in dsmt_docs:
        f.write(json.dumps(doc)+'\n')

In [22]:
import boto3
import requests

profile = "wmuser"
bucket_name = "world-modelers"

session = boto3.Session(profile_name=profile)

s3 = session.resource("s3")
s3_client = boto3.client("s3")

bucket = s3.Bucket(bucket_name)

In [23]:
download_base_path = "/Users/brandon/Desktop/DSMT-Sample-Documents/"

In [34]:
count = 0
for doc in dsmt_docs:
    s3_key = doc['stored_url'].split('.com/')[1]
    file_name = download_base_path + doc['file_name']    
    bucket.download_file(s3_key, file_name)
    count += 1
    if count % 10 == 0:
        print(count)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540


## Update documents with UAZ extractions

In [72]:
PATH_TO_EXTRACTIONS = '/Users/brandon/Downloads'
dirs = ['distrib','distrib 2','distrib 3','distrib 4']

In [73]:
files = []
for d in dirs:
    f = os.listdir(f'{PATH_TO_EXTRACTIONS}/{d}/jsonld/')
    f_ = [f'{PATH_TO_EXTRACTIONS}/{d}/jsonld/{file}' for file in f]
    files.extend(f_)

In [67]:
def load_doc(filename):
    with open(filename,'r') as f:
        j = json.loads(f.read())
        _id = j['documents'][0]['title']
        text = j['documents'][0]['text']         
        return (_id, text)

In [ ]:
count = 0
ids = set()
for f in files:
    try:
        _id, text = load_doc(f)
        if _id not in ids:
            ids.add(_id)
            doc = es.get(index, _id)['_source']
            if doc['file_type'] == '.pdf':
                doc['extracted_text']['pdfminer'] = text
            elif doc['file_type'] == '.html':
                doc['extracted_text']['html2text'] = text
            else:
                doc['extracted_text']['pdfminer'] = text
            es.index(index=index, body=doc, doc_type='_doc', id=_id)
            count += 1
            print(count)
    except Exception as e:
        print(e)
        print(f)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
'text'
/Users/brandon/Downloads/distrib/jsonld/BriefNutritionOverview.jsonld
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258

In [ ]:
query = {
    "query": {
        "exists": {
            "field": "extracted_text.pdfminer"
        }
    }
}

pdfminer_count = es.count(index=index, body=query)['count']

query = {
    "query": {
        "exists": {
            "field": "extracted_text.html2text"
        }
    }
}

html2text_count = es.count(index=index, body=query)['count']

In [ ]:
print(f"There were {pdfminer_count} docs with pdfminer extractions and {html2text_count} docs with html2text extractions")